In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
from io import BytesIO
from PIL import Image
import cv2
import re

In [3]:
true_df = pd.read_csv("small_test.csv")
true_df.head()

,Unnamed: 0,UID,growth_stage,label_quality,img_name,width,height
0,0,nwuCTQKz,4,2,nwuCTQKz.jpeg,512,147
1,7,sYLqmiXV,3,2,sYLqmiXV.jpeg,512,180
2,10,rDP2EuQN,7,2,rDP2EuQN.jpeg,512,212
3,17,dMTbnE8V,2,2,dMTbnE8V.jpeg,512,120
4,21,HQWkZRNB,2,2,HQWkZRNB.jpeg,512,159


In [4]:
subm_df = pd.read_csv("true_small.csv")
subm_df.head()

,UID,growth_stage,1,1_data,2,2_data,3,3_data,4,4_data,5,5_data
0,yu3cXCJU,3,1.0,0.971625,2.0,0.633354,3.0,0.994188,3.0,0.591444,3.0,0.980534
1,Z3vYgRMO,3,1.0,0.564363,3.0,0.999843,3.0,0.949135,3.0,0.953500,1.0,0.999506
2,2NB7F9AH,4,1.0,0.985649,3.0,0.996510,3.0,0.974284,3.0,0.999746,3.0,0.966215
3,dlDHUCGB,5,3.0,0.850931,2.0,0.987421,1.0,0.999674,6.0,0.999337,6.0,0.999953
4,f3aXpZIL,4,2.0,0.996620,3.0,0.996077,2.0,0.999680,6.0,0.998421,2.0,0.822567


In [7]:
subm_df["ground_truth"] = subm_df["UID"].apply(lambda x: np.max(true_df[true_df["UID"] == x]["growth_stage"], 0))

In [12]:
subm_df[subm_df["ground_truth"] == 6].head(20)

,UID,growth_stage,1,1_data,2,2_data,3,3_data,4,4_data,5,5_data,ground_truth


In [17]:
subm_df[subm_df["ground_truth"] - subm_df["growth_stage"] > 3].head(20)

,UID,growth_stage,1,1_data,2,2_data,3,3_data,4,4_data,5,5_data,ground_truth
37,nLKsV4O8,3,3.0,0.816828,2.0,0.979976,2.0,0.511721,2.0,0.815925,2.0,0.667813,7
124,c0qkgx48,3,4.0,0.518655,1.0,0.999175,2.0,0.913011,2.0,0.855096,1.0,0.999829,7
188,r6nkCFPd,3,3.0,0.742269,3.0,0.927819,1.0,0.792277,2.0,0.963539,2.0,0.919884,7
481,JlD2iT9k,3,6.0,0.960052,3.0,0.948286,1.0,0.976387,1.0,0.975851,1.0,0.998976,7
570,zuL2XVPm,3,3.0,0.533656,2.0,0.757258,3.0,0.983269,1.0,0.572065,3.0,0.935301,7
660,myQWGx40,3,3.0,0.590202,3.0,0.847032,1.0,0.999954,3.0,0.979699,2.0,0.551967,7
669,USGW8y1Q,3,3.0,0.966640,3.0,0.995986,1.0,0.987048,1.0,0.580775,2.0,0.996385,7
878,3jqbKWcg,3,3.0,0.962829,3.0,0.563200,2.0,0.889766,1.0,0.993305,3.0,0.996316,7
918,G6osOFZb,3,6.0,0.920580,2.0,0.623945,1.0,0.990024,1.0,0.792506,1.0,0.878623,7
1259,puAtCN69,3,3.0,0.734097,2.0,0.869435,2.0,0.920982,1.0,0.960631,3.0,0.580891,7
